<table align="center" style="text-align:center">
    <tbody>
        <tr>
            <td colspan="15"><img src="https://raw.githubusercontent.com/TomWagg/tomwagg.github.io/main/img/milky_way_dcos.png"/></td>
            <td colspan="50">
                <h1 style="font-size:50px">Get Detection Rates</h1>
                <h2 style="font-size:20px;"><i>Part of the notebook series for Wagg et al. 2021</i></h2>
                <p style="font-size:15px;">This notebook converts the raw data into detection totals and saves them for easy access.</p>
            </td>
            <td colspan="15"><img src="https://raw.githubusercontent.com/TomWagg/tomwagg.github.io/main/img/milky_way_dcos.png"/></td>
        </tr>
    </tbody>
</table>

In [1]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [3]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"
simple_sim_folder = data_path + "simple_mw_simulation/"
extended_sim_folder = data_path + "simulation_10yr/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")
total_DCOs_in_simple_MW = np.load(data_path + "total_DCO_in_simple_MW.npy")

total_DCOs_in_simple_MW_new = np.zeros_like(total_DCOs_in_MW)
total_DCOs_in_simple_MW_new[:, 0] = total_DCOs_in_simple_MW[0]
total_DCOs_in_simple_MW_new[:, 5] = total_DCOs_in_simple_MW[1]
total_DCOs_in_simple_MW = total_DCOs_in_simple_MW_new

# Calculate number of detections

In [7]:
def get_detections(dco_types=dco_types, variation_list=range(len(variations)), folder=sim_folder, totals=total_DCOs_in_MW, extended_mission=False):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))

    for d in range(len(dco_types)):
        for v in variation_list:
            # open the proper output file
            fname = folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            
            if extended_mission:
                fname = folder + "{}_{}_10yr_all.h5".format(dco_types[d], variations[v]["file"])
            
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    parameter = "n_detect"
                    n_detect = f["simulation"].attrs[parameter].astype(int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_detect))
                
                checks = np.zeros(len(n_detect))
                for i in range(len(n_detect)):
                    weights = full_data["weight"][cursor:cursor + n_detect[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights) / total_mw_weight[i] * totals[d][v]
                    cursor += n_detect[i]

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
                
            print(d, v, detections[d][v].mean())
    
    return detections

In [8]:
detections_4yr = get_detections()

0 0 73.80783369032252
0 1 68.62366120385612
0 2 46.84862548763242
0 3 46.779646198353895
0 4 65.84109988197558
0 5 153.6305161178988
0 6 27.821580433662557
0 7 58.11973117813617
0 8 67.36739038755559
0 9 26.62016179861819
0 10 151.06891782969865
0 11 50.239850558539594
0 12 95.89324804919633
0 13 58.035859509547265
0 14 75.08305257718563
0 15 82.38948931797887
0 16 91.55351131976154
0 17 89.38411729509289
0 18 75.39496679533205
0 19 5.694392047082277
1 0 42.213972596870185
1 1 22.3072577369226
1 2 8.22970777718762


/var/folders/3c/rntdf51j7jldynz_vkv0_3ww0000gn/T/ipykernel_40607/2404253287.py:27: RuntimeWarning: invalid value encountered in double_scalars
  checks = np.sum(weights)**2 / np.sum(weights**2)


1 3 7.420886798563629
1 4 20.99500531062175
1 5 197.61074728432706
1 6 2.096371145504264
1 7 21.624367326569494
1 8 37.81348407467861
1 9 16.293546710261513
1 10 56.63888254339634
1 11 70.0094390048155
1 12 29.97284123489926
1 13 51.76866757421958
1 14 43.20473362682169
1 15 86.3005802375068
1 16 142.60713205454445
1 17 108.8317922172436
1 18 33.95393881388039
1 19 15.486080721460935
2 0 7.9421002667664
2 1 2.895727185379542
2 2 3.9818298893804216
2 3 12.723437395199408
2 4 2.750812152758768
2 5 16.896838100955957
2 6 10.14040141837234
2 7 5.866511286471199
2 8 27.61721257533844
2 9 34.83348405731635
2 10 9.911149733723445
2 11 6.652564900516423
2 12 7.169384745014485
2 13 8.26159484261536
2 14 7.997393322720041
2 15 15.353282959305751
2 16 33.74133289471605
2 17 7.241657269176164
2 18 8.77644775013363
2 19 8.45776051387555


In [9]:
detections_10yr = get_detections(dco_types, extended_mission=True, folder=extended_sim_folder)

0 0 117.49470260871541
0 1 107.51826461157428
0 2 75.61640666419683
0 3 71.02261684700977
0 4 100.70227413500433
0 5 238.43116608032045
0 6 43.72366857168408
0 7 91.35245057613889
0 8 109.23750208010925
0 9 42.236225751973805
0 10 228.98004295286518
0 11 76.38406148583994
0 12 153.18091716713434
0 13 91.22163084479088
0 14 120.12407924603669
0 15 129.6764554217308
0 16 142.49787851401393
0 17 140.0598067077698
0 18 111.99384045673665
0 19 9.225239692914757
1 0 71.05713214946786
1 1 36.710781485866626
1 2 13.561604759657962
1 3 12.117263183209998
1 4 28.869572005669873
1 5 288.8120806199364
1 6 3.4712578177694144
1 7 34.655480402700896
1 8 62.49096465215815
1 9 26.54569112695937
1 10 95.76031212376435
1 11 117.05887925102495
1 12 49.675283154488824
1 13 84.58479305590787
1 14 72.05016560626514
1 15 144.62328270575864
1 16 228.59677142172112
1 17 180.03640094744804
1 18 55.704642204484855
1 19 26.107842309990122
2 0 13.036639307270876
2 1 4.685194896342399
2 2 6.510130602611694
2 3 21.15

In [10]:
detections_simple = get_detections(folder=simple_sim_folder + "simple_mw_", variation_list=[0, 5], totals=total_DCOs_in_simple_MW)

0 0 52.3751446651769
0 5 96.72893783753781
1 0 24.93725218656574
1 5 101.39867208989773
2 0 17.374324879667842
2 5 43.457652745656496


In [11]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)